In [28]:
import pandas as pd

df1=pd.read_csv('balanced_urls.csv')
new_df1=df1.drop(columns="result")
new_df1

,url,label
0,https://www.google.com,benign
1,https://www.youtube.com,benign
2,https://www.facebook.com,benign
3,https://www.baidu.com,benign
4,https://www.wikipedia.org,benign
...,...,...
632503,xbox360.ign.com/objects/850/850402.html,malicious
632504,games.teamxbox.com/xbox-360/1860/Dead-Space/,malicious
632505,www.gamespot.com/xbox360/action/deadspace/,malicious
632506,en.wikipedia.org/wiki/Dead_Space_(video_game),malicious


In [29]:
df2=pd.read_csv('malicious_phish.csv')
df2

,url,label
0,stadiumpage.blogspot.com/2010/02/unrealized-co...,benign
1,johnnyklister.com/,benign
2,imdb.com/company/co0172690/,benign
3,http://www.arjanland-kerstbomen.nl/snijgroen,defacement
4,liquida.com/playhouse-disney-channel-games/,benign
...,...,...
4048,senatorsextra.com/main/ottawa-senators-prospec...,benign
4049,fareastvibes.com/?p=679,benign
4050,http://themeforest.net/item/shop7-ecommerce-em...,benign
4051,youtube.com/watch?v=2rJvmDdFnmI,benign


In [20]:
# Map the values in the 'type' column
df2['label'] = df2['label'].replace({'phishing': 'malicious', 'defacement': 'malicious', 'malware': 'malicious'})


In [21]:
df2_renamed = df2.rename(columns={'type': 'label'})
df2_renamed

,url,label
0,stadiumpage.blogspot.com/2010/02/unrealized-co...,benign
1,johnnyklister.com/,benign
2,imdb.com/company/co0172690/,benign
3,http://www.arjanland-kerstbomen.nl/snijgroen,malicious
4,liquida.com/playhouse-disney-channel-games/,benign
...,...,...
4048,senatorsextra.com/main/ottawa-senators-prospec...,benign
4049,fareastvibes.com/?p=679,benign
4050,http://themeforest.net/item/shop7-ecommerce-em...,benign
4051,youtube.com/watch?v=2rJvmDdFnmI,benign


In [22]:
combined_df = pd.concat([new_df1, df2_renamed], ignore_index=True)
combined_df = combined_df.drop_duplicates(subset='url')
combined_df.to_csv('big_data.csv',index=False)

In [23]:
import pandas as pd

# Assuming combined_df is your DataFrame
combined_df['label'] = combined_df['label'].map({'benign': 0, 'malicious': 1})
combined_df

,url,label
0,https://www.google.com,0
1,https://www.youtube.com,0
2,https://www.facebook.com,0
3,https://www.baidu.com,0
4,https://www.wikipedia.org,0
...,...,...
636556,senatorsextra.com/main/ottawa-senators-prospec...,0
636557,fareastvibes.com/?p=679,0
636558,http://themeforest.net/item/shop7-ecommerce-em...,0
636559,youtube.com/watch?v=2rJvmDdFnmI,0


In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Extract features from the 'url' column using TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=500)
X = vectorizer.fit_transform(combined_df['url'])

# Target labels
y = combined_df['label']


In [25]:
from sklearn.model_selection import train_test_split

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize the SVM model
model = SVC(kernel='rbf', random_state=42)

# Train the SVM model
model.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Evaluate the model
print(classification_report(y_test, y_pred))



In [ ]:
def predict_url(url):
    # Convert the URL to feature vector using the trained vectorizer
    url_vector = vectorizer.transform([url])

    # Predict the label (0: benign, 1: malicious)
    prediction = model.predict(url_vector)
    
    return 'benign' if prediction == 0 else 'malicious'

# Test with a new URL
new_url = "http://example.com"
result = predict_url(new_url)
print(f"The URL '{new_url}' is predicted as: {result}")


In [ ]:
from sklearn.model_selection import GridSearchCV

# Define hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
}

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=RandomForestClassifier(), param_grid=param_grid, cv=3, scoring='accuracy')

# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters
print(f"Best Parameters: {grid_search.best_params_}")

# Train the model with the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)


In [26]:
from PIL import Image  # Import Image from the PIL library
from pyzbar.pyzbar import decode  # Import decode from pyzbar

def extract_qr_code_data(image_path):
    """
    Décode un QR code depuis une image et retourne toutes les informations décodées.

    :param image_path: Chemin de l'image contenant le QR code.
    :return: Liste des données décodées ou une liste vide si aucun QR code valide n'est détecté.
    """
    try:
        # Charger l'image
        image = Image.open(image_path)
        # Décoder le QR code
        decoded_data = decode(image)
        if decoded_data:
            # Retourne une liste des données décodées sous forme de chaîne de caractères
            return [obj.data.decode('utf-8').strip() for obj in decoded_data]
        else:
            return []  # Aucun QR code valide détecté
    except Exception as e:
        print(f"Erreur lors de l'ouverture ou du traitement de l'image : {e}")
        return []

# Test with the image paths
image_path = r"C:\Users\Soso\Downloads\balanced_urls.csv\Multi-version QR codes dataset\malicious\malicious_version_7_416263.png"
image_path1 = "Multi-version QR codes dataset/benign/benign_version_6_100032"

result = extract_qr_code_data(image_path)
print(result)


['416263    http://islapartmentandhotel.com/www.paypal.com...\nName: url, dtype: object']


In [27]:
def find_url_in_list(data_list):
    for element in data_list:
        if "http" in element:
            return element  # Retourne l'élément contenant "http"
    return None  # Retourne None si aucun élément ne contient "http"
for elem in result:
    url = find_url_in_list(elem.split(" "))
url

'http://islapartmentandhotel.com/www.paypal.com...\nName:'